## Setup

Import necessary modules.

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig
import os

## Initialize Gemini Client

In [ ]:
# Initialize Gemini client
client = genai.Client()

# Model configuration
MODEL_NAME = "gemini-2.0-flash-exp"
TEMPERATURE = 0.0
MAX_TOKENS = 2048

## Example Question and Re-ranked Documents

Assuming you've already done retrieval and re-ranking from the previous notebook.

In [ ]:
question = "Hei, jeg lurer på når man slipper å betale dokumentavgift hvis bolig eller eiendom overføres, for eksempel hvis samboere går fra hverandre eller en av dem dør, hvordan det er hvis man arver fast eiendom, og om det også gjelder dersom man får eiendom gjennom testament ved privat skifte?"

print("Spørsmål:", question)

In [ ]:
# Example: re-ranked documents from previous notebook
# In practice, these would come from your ranking.ipynb workflow

ranked_docs = [
    {
        "id": "doc_1",
        "content": "Dokumentavgift skal ikke betales ved tinglysing av dokumenter som gjelder arv eller skifte av dødsbo, herunder privat skifte. Dette følger av dokumentavgiftsloven § 8.",
        "source": "dokumentavgift-2025.md",
        "rerank_score": 0.89
    },
    {
        "id": "doc_2",
        "content": "Fritaksbestemmelsen gjelder både ved arv etter lov og ved testament. Dette omfatter også privat skifte av dødsbo. Overføring av eiendom som ledd i arv er derfor fritatt for dokumentavgift.",
        "source": "dokumentavgift-2025.md",
        "rerank_score": 0.85
    },
    {
        "id": "doc_3",
        "content": "Ved overdragelse mellom samboere gjelder de ordinære reglene for dokumentavgift, med mindre overdragelsen skjer som følge av dødsfall. Ved samlivsbrudd mellom samboere må det normalt betales dokumentavgift.",
        "source": "veiledning_dokumentavgift.pdf",
        "rerank_score": 0.82
    },
    {
        "id": "doc_4",
        "content": "Når en samboer dør, kan den gjenlevende samboeren ha rett til å overta bolig etter reglene i arveloven. Slik overdragelse vil være fritatt for dokumentavgift dersom den skjer som ledd i arveoppgjøret.",
        "source": "veiledning_dokumentavgift.pdf",
        "rerank_score": 0.79
    },
    {
        "id": "doc_5",
        "content": "Testament ved privat skifte følger samme regler som ordinær arv når det gjelder dokumentavgift. Det betyr at eiendom som overføres gjennom testament ved privat skifte er fritatt for dokumentavgift.",
        "source": "dokumentavgift-2025.md",
        "rerank_score": 0.77
    }
]

print(f"Har {len(ranked_docs)} re-rankede dokumenter")

## Format Documents for Context

We need to format the ranked documents into a context string that the LLM can use.

In [ ]:
def format_docs_for_context(docs: list[dict]) -> str:
    """
    Format ranked documents into a context string.
    Groups by source and includes content.
    """
    from collections import defaultdict
    
    # Group by source
    grouped = defaultdict(list)
    for doc in docs:
        source = doc.get("source", "Ukjent kilde")
        content = doc.get("content", "")
        grouped[source].append(content)
    
    # Build formatted sections
    sections = []
    for source, contents in grouped.items():
        section = f"[Kilde: {source}]\n"
        section += "\n'''\n".join(contents)
        sections.append(section)
    
    return "\n\n".join(sections)


# Format the documents
context = format_docs_for_context(ranked_docs)

print("Formatert kontekst:")
print("=" * 80)
print(context[:500] + "...")

## Build System Prompt

Create a system prompt that instructs the model how to use the context.

In [ ]:
system_prompt = """
Du er en ekspert på norsk dokumentavgift og skal hjelpe brukere med å forstå regelverket.

Bruk informasjonen nedenfor til å svare presist og korrekt på brukerens spørsmål.
Hvis informasjonen ikke er tilstrekkelig til å svare, si det tydelig.

Svar på norsk (bokmål) og strukturer svaret tydelig med:
- Klare punkt for hver situasjon brukeren spør om
- Konkrete svar på om dokumentavgift må betales eller ikke
- Relevante referanser til lovverk når det er hensiktsmessig
""".strip()

print("System prompt:")
print(system_prompt)

## Construct Full Prompt

Combine system prompt, context, and user question into a complete prompt.

In [ ]:
def construct_prompt(system_prompt: str, context: str, question: str) -> str:
    """
    Construct the full prompt for the LLM.
    """
    full_prompt = f"""{system_prompt}

Kilder:
{context}

Spørsmål:
```{question}```
"""
    return full_prompt


full_prompt = construct_prompt(system_prompt, context, question)

print("Full prompt (første 800 tegn):")
print("=" * 80)
print(full_prompt[:800] + "...")

## Generate Answer

Send the prompt to Gemini and get the answer.

In [ ]:
def generate_answer(
    client: genai.Client,
    prompt: str,
    model: str = MODEL_NAME,
    temperature: float = TEMPERATURE,
    max_tokens: int = MAX_TOKENS,
) -> str:
    """
    Generate answer using Gemini.
    """
    response = client.models.generate_content(
        model=model,
        contents=[{"role": "user", "parts": [{"text": prompt}]}],
        config=GenerateContentConfig(
            temperature=temperature,
            max_output_tokens=max_tokens,
            candidate_count=1,
        ),
    )
    
    if not response.candidates:
        return "Ingen svar generert."
    
    return response.candidates[0].content.parts[0].text


# Generate the answer
print("Genererer svar...")
answer = generate_answer(client, full_prompt)

print("\n" + "=" * 80)
print("SVAR:")
print("=" * 80)
print(answer)

## Complete Pipeline Function

Here's a complete function that combines all steps.

In [ ]:
def generate_rag_answer(
    question: str,
    ranked_docs: list[dict],
    client: genai.Client,
    system_prompt: str,
    model: str = MODEL_NAME,
    temperature: float = TEMPERATURE,
    max_tokens: int = MAX_TOKENS,
) -> dict:
    """
    Complete RAG answer generation pipeline.
    
    Args:
        question: User's question
        ranked_docs: List of re-ranked documents
        client: Gemini client
        system_prompt: System instructions
        model: Model name
        temperature: Generation temperature
        max_tokens: Maximum output tokens
    
    Returns:
        Dict with answer, prompt, and sources
    """
    # Format context
    context = format_docs_for_context(ranked_docs)
    
    # Build prompt
    full_prompt = construct_prompt(system_prompt, context, question)
    
    # Generate answer
    answer = generate_answer(
        client=client,
        prompt=full_prompt,
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    
    return {
        "answer": answer,
        "question": question,
        "full_prompt": full_prompt,
        "sources": ranked_docs,
        "num_sources": len(ranked_docs),
    }


# Test the complete pipeline
result = generate_rag_answer(
    question=question,
    ranked_docs=ranked_docs,
    client=client,
    system_prompt=system_prompt,
)

print("\n" + "=" * 80)
print("RAG PIPELINE RESULTAT")
print("=" * 80)
print(f"\nSpørsmål: {result['question']}")
print(f"\nAntall kilder: {result['num_sources']}")
print(f"\nSvar:\n{result['answer']}")

## Try Different Temperatures

Let's see how temperature affects the answer quality.

In [ ]:
temperatures = [0.0, 0.3, 0.7]

for temp in temperatures:
    print(f"\n{'=' * 80}")
    print(f"TEMPERATURE: {temp}")
    print("=" * 80)
    
    result = generate_rag_answer(
        question=question,
        ranked_docs=ranked_docs,
        client=client,
        system_prompt=system_prompt,
        temperature=temp,
    )
    
    print(result['answer'][:500] + "...\n")

## Summary

### Key Components of Answer Generation

1. **Context Formatting**: Group and format ranked documents clearly
2. **System Prompt**: Give clear instructions on how to use the context
3. **Prompt Construction**: Combine all elements into a coherent prompt
4. **Generation**: Use appropriate model parameters (temperature, max_tokens)

### Best Practices

- **Use top N documents**: Typically 5-10 re-ranked documents provide enough context
- **Low temperature**: Use 0.0-0.3 for factual, grounded answers
- **Clear formatting**: Use delimiters (```, ''', etc.) to separate sections
- **Source attribution**: Include source information in context for transparency

### Complete RAG Pipeline

1. **Query rewriting** (optional): Rephrase question for better retrieval
2. **Vector search**: Get candidate documents (20-100)
3. **Re-ranking**: Rank by relevance (keep top 5-10)
4. **Answer generation**: Use ranked docs as context for LLM
5. **Evaluation**: Assess relevance and groundedness